Atlas Kiti Uygulama Scripti 

In [0]:
#V0.1 - Eğitilmiş RNN modelini çalıştırabilen bir başlangıç kodu yazıldı.
#    -csv ve txt şeklinde derlenmiş dataların işlenmesi destekleniyor.
#    -Google Colab platformunda teste hazır.
#    -Github'da bulunan örnek datalar ve modeller kullanılabilir veya kullanıcılar 
#       eğitim ve crawler scriptlerini kullanarak kendi girdilerini oluşturabilir.  
##############################################################################

#coding:utf8

# Kütüphanelerin içe aktarılması 
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
plt.style.use('dark_background')
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from keras.models import Sequential
from keras.models import load_model
from keras.layers import Dense, Dropout, Embedding, LSTM, GlobalMaxPooling1D, SpatialDropout1D
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder


#Google Colab ve Drive integrasyonu. Kullanıcılar kendi yerel bilgisayarlarında çalıştırmak için model ve data path değişkenlerini değiştirmeli.
from google.colab import drive
drive.mount('/content/gdrive')
model_path = '/content/gdrive/My Drive/hackathon/RNN12kdataES.h5'
data_path = '/content/gdrive/My Drive/hackathon/testData.txt'

max_words = 8192
#max_phrase_len eğitim scriptinde belirlenen ve buraya hard-coded girilen bir değer model değiştiğinde bu da değiştirilmeli
max_phrase_len = 593


# Verinin giriş ve çıkış olarak ayrılması
dataframe = pd.read_csv(data_path, sep='|')

dataframe.Comment=dataframe.Comment.astype(str)
dataframe.Score=dataframe.Score.astype(float)


input_val = dataframe['Comment']
output_val = dataframe['Score']

# Modelin yüklenmesi ve detayı
model_atlas = load_model(model_path)
model_atlas.summary()



Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
Model: "sequential_21"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_21 (Embedding)     (None, 593, 256)          2097152   
_________________________________________________________________
spatial_dropout1d_21 (Spatia (None, 593, 256)          0         
_________________________________________________________________
lstm_21 (LSTM)               (None, 256)               525312    
_________________________________________________________________
dense_41 (Dense)             (None, 256)               65792     
_________________________________________________________________
dropout_21 (Dropout)         (None, 256)               0         
_________________________________________________________________
dense_42 (Dense)             (None, 7)                

In [0]:
# Bilgisayara girdi verirken cümlelerin tokenize edilmesi.
tokenizer = Tokenizer(
    num_words = max_words,
    filters = '"#$%&()*+-/:;<=>@[\]^_`{|}~')
tokenizer.fit_on_texts(input_val)
input_val = tokenizer.texts_to_sequences(input_val)
input_val = pad_sequences(input_val, maxlen = max_phrase_len)


# Çıkış değerleri sistem tarafından değerlendirilebilmesi için OneHotEncoder formatına çevrilir
output_val = np.array(output_val)
label_encoder = LabelEncoder()
output_val_intencoded = label_encoder.fit_transform(output_val)
onehot_encoder = OneHotEncoder(sparse=False)
output_val_intencoded = output_val_intencoded.reshape(len(output_val_intencoded), 1)
output_val = onehot_encoder.fit_transform(output_val_intencoded)




/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


In [0]:
# Her bir girdinin çıktısının hangi sınıfa/etikete ait olduğunu olasıksal olarak verir
PredictedValueWeights = model_atlas.predict([input_val])

# Verilerin girdileri ile birlikte çıktıları da verilerek modelin performansı ölçülebilir.
results = model_atlas.evaluate(input_val, output_val, verbose=1, batch_size=128)
print(results[1]) 


100/100 [==============================] - 2s 21ms/step
0.7200000286102295
